In [3]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# RQ3: How stable is the community working on the topics (new authors emerging, authors staying on for the whole time,
# or authors “leaving” the area)?

In [5]:
data = pd.read_csv('./data/500se.csv')
data.head()

,Cites,Authors,Title,Year,Source,Publisher,ArticleURL,CitesURL,GSRank,QueryDate,...,Volume,Issue,StartPage,EndPage,ECC,CitesPerYear,CitesPerAuthor,AuthorCount,Age,Abstract
0,7653,"C Wohlin, P Runeson, M Höst, MC Ohlsson, B Reg...",Experimentation in software engineering,2012,NaN,books.google.com,https://books.google.com/books?hl=en&lr=&id=QP...,https://scholar.google.com/scholar?cites=11975...,1,2021-05-11 13:03:30,...,NaN,NaN,NaN,NaN,7653,850.33,1276,6,9,Like other sciences and engineering discipline...
1,441,R Mall,Fundamentals of software engineering,2018,NaN,books.google.com,https://books.google.com/books?hl=en&lr=&id=-J...,https://scholar.google.com/scholar?cites=22762...,2,2021-05-11 13:03:30,...,NaN,NaN,NaN,NaN,441,147.00,441,1,3,"This new edition of the book, is restructured ..."
2,698,P Jalote,An integrated approach to software engineering,2012,NaN,books.google.com,https://books.google.com/books?hl=en&lr=&id=Hi...,https://scholar.google.com/scholar?cites=65505...,3,2021-05-11 13:03:30,...,NaN,NaN,NaN,NaN,698,77.56,698,1,9,A lot has changed in the fast-moving area of s...
3,1051,"N Juristo, AM Moreno",Basics of software engineering experimentation,2013,NaN,books.google.com,https://books.google.com/books?hl=en&lr=&id=iJ...,https://scholar.google.com/scholar?cites=81867...,4,2021-05-11 13:03:30,...,NaN,NaN,NaN,NaN,1051,131.38,526,2,8,Basics of Software Engineering Experimentation...
4,491,"NMA Munassar, A Govardhan",A comparison between five models of software e...,2010,International Journal of Computer Science …,Citeseer,http://citeseerx.ist.psu.edu/viewdoc/download?...,https://scholar.google.com/scholar?cites=70840...,5,2021-05-11 13:03:30,...,NaN,NaN,NaN,NaN,491,44.64,246,2,11,This research deals with a vital and important...


In [6]:
def CreateTermDocMatrix(column):

    count_vectorizer = CountVectorizer(stop_words='english')
    #count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(data[column].values.astype('U'))

    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names())
    df['PublishingYear'] = data['Year']
    returndf = df.groupby('PublishingYear').sum().transpose()
    return returndf

In [7]:
#d = CreateTermDocMatrix('Authors')

#d.loc[d[2017] > 3] 
author_year_df = pd.DataFrame(data, columns=['Authors', 'Year'])

data['count'] = 1
result = data.pivot_table(index='Authors', 
                        columns='Year', 
                        values='Cites',
                        aggfunc=np.sum,
                        fill_value = 0)
#df = author_year_df.pivot(index='Authors', columns='Year', values='count')
#result = result.fillna(0,inplace=True)

result

Year,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Authors,,,,,,,,,,,,,
"A Adewumi, S Misra, N Omoregbe",0,0,0,0,37,0,0,0,0,0,0,0,0
"A Agrawal, W Fu, T Menzies",0,0,0,0,0,0,0,0,0,111,0,0,0
"A Aitken, V Ilango",0,0,0,0,82,0,0,0,0,0,0,0,0
"A Ampatzoglou, S Bibi, P Avgeriou, M Verbeek…",0,0,0,0,0,0,0,0,0,0,40,0,0
"A Arcuri, L Briand",0,0,756,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Z Li, XY Jing, X Zhu",0,0,0,0,0,0,0,0,0,65,0,0,0
"Z Mahmood, S Saeed",0,0,0,0,37,0,0,0,0,0,0,0,0
"Z Sharafi, T Shaffer, B Sharif…",0,0,0,0,0,0,62,0,0,0,0,0,0


In [8]:
#data['count'] = 1
#data.reset_index().drop_duplicates(subset='Authors',keep='last').set_index('Authors')
#df = data.pivot(index='Authors', columns='Year', values='count')
#df

In [88]:
data1 = pd.read_csv('./data/export1.csv')
data2 = pd.read_csv('./data/export2.csv')
data = pd.concat([data1,data2])

In [89]:
data.keys()

Index(['Document Title', 'Authors', 'Author Affiliations', 'Publication Title',
       'Date Added To Xplore', 'Publication Year', 'Volume', 'Issue',
       'Start Page', 'End Page', 'Abstract', 'ISSN', 'ISBNs', 'DOI',
       'Funding Information', 'PDF Link', 'Author Keywords', 'IEEE Terms',
       'INSPEC Controlled Terms', 'INSPEC Non-Controlled Terms', 'Mesh_Terms',
       'Article Citation Count', 'Patent Citation Count', 'Reference Count',
       'License', 'Online Date', 'Issue Date', 'Meeting Date', 'Publisher',
       'Document Identifier'],
      dtype='object')

In [90]:
year = data['Publication Year']
data = data['Authors'].str.split(';', expand=True)
data = data.fillna(value=1)

In [91]:
def GetAuthorsandYear(data):
    b = None
    for column in data:
        r = data[column].to_frame().join(year)
        r.rename(columns={ r.columns[0]: "Author" }, inplace = True)
        if b is None:
            b = r
        else:
            b = pd.concat([b, r])
    b.reset_index(drop = True)
    b = b[b.Author != 1]
    b['Count'] = 1
    b = b.rename({ 'Publication Year': 'Year'}, axis=1)
    result = b.pivot_table(index='Author', 
                        columns='Year', 
                        values='Count',
                        aggfunc=np.sum,
                        fill_value = 0)
    return result

In [92]:
x = GetAuthorsandYear(data)
x

Year,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Author,,,,,,,,,,,,,
A. -F. Zhu,1,0,0,0,0,0,1,0,0,0,0,0,0
A. A. Abbasi,0,0,0,1,0,0,0,0,0,0,0,0,1
A. A. Tsintzira,1,0,0,0,0,0,0,0,0,0,0,0,1
A. Abdin,1,0,0,0,0,0,0,0,0,1,0,0,0
A. Agah,0,0,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z. Zhi,1,0,0,0,0,0,0,0,0,0,0,1,0
Z. Zhou,1,0,0,0,0,0,0,0,0,0,0,0,0
Z. Zhu,2,0,0,0,0,1,0,0,0,0,1,0,0


In [93]:
def SortbyYear(data,jahr,publications):    
     for item in data:   
            p = x.loc[x[jahr] >= publications]
            return p

In [95]:
SortbyYear(x,2021,3)

Year,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Author,,,,,,,,,,,,,
A. E. Hassan,43,2,2,0,2,2,3,6,4,2,9,12,7
C. Tantithamthavorn,5,0,0,0,0,0,0,0,0,0,0,2,3
C. Wang,20,0,0,0,1,0,2,1,1,2,4,5,3
D. Lo,27,1,0,1,0,0,0,1,1,1,11,10,5
E. Shihab,10,0,0,1,1,0,1,0,2,0,1,2,4
F. Palomba,6,0,0,0,0,0,0,0,1,1,0,1,3
F. Sarro,7,0,0,0,0,0,0,0,1,0,1,1,4
G. Peng,4,0,0,0,0,0,1,0,0,0,0,0,3
H. Li,19,0,0,0,0,2,1,1,3,2,3,2,4


Int64Index([2018, 2019, 2020, 2021], dtype='int64', name='Year')

0          T. Dingsøyr
1                G. Li
2             M. Ardis
3            O. Hazzan
4           P. Johnson
             ...      
1941    P. Abrahamsson
1942               NaN
1943               NaN
1944               NaN
1945           P. Lago
Name: 0, Length: 1946, dtype: object

,,,,,,,,,Publication Year,2012
0,1,2,3,4,5,6,7,8,9,
D. J. Crichton,C. A. Mattmann,L. Cinquini,A. Braverman,D. Waliser,M. Gunson,A. F. Hart,C. E. Goodale,P. Lean,J. Kim,1
